## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-14-05-42-05 +0000,nyt,"Sharon Camp, Mother of the ‘Plan B’ Contracept...",https://www.nytimes.com/2025/11/14/health/poli...
1,2025-11-14-05-37-00 +0000,wsj,China Registers Worst Investment Decline in Ye...,https://www.wsj.com/world/china/chinas-economi...
2,2025-11-14-05-35-02 +0000,nyt,Of Books and Men,https://www.nytimes.com/2025/11/14/world/booke...
3,2025-11-14-05-20-23 +0000,bbc,Who are the released hostages?,https://www.bbc.com/news/articles/cpvl9k4mw8no...
4,2025-11-14-05-14-30 +0000,nyt,A.I. Cheating Rattles Top Universities in Sout...,https://www.nytimes.com/2025/11/14/world/asia/...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
26,trump,35
75,new,20
372,epstein,15
32,shutdown,15
196,california,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
333,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...,106
223,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...,98
203,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...,81
251,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...,78
233,2025-11-13-15-27-26 +0000,bbc,What we know about new Epstein emails that men...,https://www.bbc.com/news/articles/c3rj0d97ynvo...,77


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
333,106,2025-11-13-06-05-00 +0000,wsj,President Trump signed a spending package to r...,https://www.wsj.com/politics/policy/house-spen...
151,58,2025-11-13-20-23-00 +0000,wsj,The Justice Department is suing California ove...,https://www.wsj.com/politics/policy/california...
301,46,2025-11-13-10-30-00 +0000,wsj,Heard on the Street: Borrowing from a builder ...,https://www.wsj.com/economy/housing/builders-c...
203,46,2025-11-13-17-18-34 +0000,cbc,Trump-appointed attorneys have been disqualifi...,https://www.cbc.ca/news/world/us-va-halligan-e...
213,44,2025-11-13-16-58-27 +0000,nypost,Disney shares plunge 9% as ABC’s ‘World News T...,https://nypost.com/2025/11/13/business/disney-...
251,43,2025-11-13-13-58-49 +0000,nypost,Trump-obsessed writer Michael Wolff encouraged...,https://nypost.com/2025/11/13/us-news/michael-...
223,40,2025-11-13-16-08-00 +0000,wsj,President Trump’s plan to allow oil drilling o...,https://www.wsj.com/politics/policy/trumps-off...
21,39,2025-11-14-03-37-55 +0000,nypost,NYC Mayor-elect Zohran Mamdani holds secretive...,https://nypost.com/2025/11/13/us-news/nyc-mayo...
18,37,2025-11-14-03-56-00 +0000,nypost,Devastating blaze rips through historic Long I...,https://nypost.com/2025/11/13/us-news/devastat...
210,32,2025-11-13-17-01-23 +0000,nypost,Rep. Anna Paulina Luna launches probe into wil...,https://nypost.com/2025/11/13/us-news/rep-anna...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
